In [1]:
import time
import torch
import torchvision
from torchvision.models import mobilenet_v3_small
from fastai.vision.models.unet import DynamicUnet
from fastai.vision.learner import create_body
from torch import nn
import gc
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
def calculate_time(model,ip,batch,cuda):
    if(cuda):
        model=model.to('cuda')
        ip=ip.to('cuda')
        print(f'Using cuda\n')

    start_time = 0
    end_time = 0

    # Avoiding init time
    '''
    we run some dummy examples through the network to do a ‘GPU warm-up.’ 
    This will automatically initialize the GPU and prevent it from going into power-saving mode 
    when we measure time. (https://deci.ai/blog/measure-inference-time-deep-neural-networks/)
    '''
    
    with torch.inference_mode():
        _ = model(ip) # GPU WARMUP
    
    with torch.inference_mode():
        start_time = time.perf_counter()
        _ = model(ip*5)                       # Multiplied by 5 to make sure old results aren't cached and just returned
        end_time = time.perf_counter()
    
    duration = end_time - start_time
    
    # Print the duration in seconds
    print(f"Duration: {duration} seconds\nFPS: {batch/duration:.0f}")

    del ip
    del model
    del _
    gc.collect()
    if(cuda):
        torch.cuda.empty_cache()

In [3]:
def build_mobile_unet(n_input=3, n_output=3, size=224):
    body = create_body(torchvision.models.mobilenet_v3_small(), n_in=n_input, pretrained=True, cut=-2)
    net_G = DynamicUnet(body, n_output, (size, size))
    return net_G

In [4]:
model_d=mobilenet_v3_small()
model_d.classifier[3]=nn.Linear(in_features=1024,out_features=2,bias=True)

model_g=build_mobile_unet()

#### Detection - method 1 (direct)

In [5]:
model_d.load_state_dict(torch.load('method1(0.974).pt'))
calculate_time(model_d,torch.rand((1000,3,224,224)),1000,True)

Using cuda

Duration: 0.0045580290002362744 seconds
FPS: 219393


#### Detection - method 2 (edge detection)

In [6]:
model_d.load_state_dict(torch.load('method2(0.990).pt'))
calculate_time(model_d,torch.rand((1000,3,224,224)),1000,True)

Using cuda

Duration: 0.0038775150001129077 seconds
FPS: 257897


#### Generator

In [7]:
calculate_time(model_g,torch.rand((16,3,224,224)),16,True)

Using cuda



OutOfMemoryError: CUDA out of memory. Tried to allocate 1.73 GiB. GPU 0 has a total capacty of 5.80 GiB of which 572.69 MiB is free. Process 13571 has 766.00 MiB memory in use. Process 13854 has 766.00 MiB memory in use. Including non-PyTorch memory, this process has 3.73 GiB memory in use. Of the allocated memory 3.56 GiB is allocated by PyTorch, and 18.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF